# pseudocode/what I need to write

- I load the dataset with erroneous labels
- process the dataset by applying a template and tokenizing it with target tokens being the labels
    - make sure padding/truncation and batching is done correctly, so target token is really the label
- feed these encodings into the pytorch Trainer somehow
- compute metrics:
    - f1, accuracy on test
    - f1, accuracy on erroneous subset of test
    - f1, accuracy on non-erroneous subset of test
- incorporate wandb

- convert this notebook into a python file so I can run more of these experiments more efficiently

In [29]:
from transformers import AutoTokenizer
import torch

model_name = "gpt2"

# ds_name = "imdb"
ds_name = "./custom-datasets/imdb_erroneous"
template = "{}\n\nIs the above review positive?\n\n"
verbalizers = ["no", "yes"]  # need to be one token


max_length = 1024
lr = 1e-3
num_epochs = 1
batch_size = 8
weight_decay = 0.001

device = "cuda"

cfg = {
    "learning_rate": lr,
    "weight_decay": weight_decay,
    "batch_size": batch_size,
    "max_tokens": max_length,
    "ds_name": ds_name,
    "model_name": model_name,
    "num_epochs": num_epochs,
    "verbalizers": verbalizers,
    "template": template,
}

In [30]:
from datasets import load_dataset, DatasetDict, load_from_disk
from torch.utils.data import DataLoader
from transformers import default_data_collator

# load dataset
first, second = ds_name.split(":") if ":" in ds_name else (ds_name, None)
# ds = load_dataset(first, second)
ds = load_from_disk(ds_name)
ds

DatasetDict({
    train: Dataset({
        features: ['text', 'label', 'true_label'],
        num_rows: 25000
    })
    test: Dataset({
        features: ['text', 'label', 'true_label'],
        num_rows: 25000
    })
})

In [19]:

ds["train"] = ds["train"].shuffle()
ds["test"] = ds["test"].shuffle()

ds = DatasetDict({
    "train": ds["train"].select(range(10_000)),
    "validation": ds["train"].select(range(10_000, 11_000)),
    "test": ds["test"].select(range(1000))
})

In [20]:
# instantiate tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name)
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

# define templatize and tokenize functions
def tokenize_examples(examples):
    batch_size = len(examples["text"])
    print(batch_size)

    # apply template to each example
    texts = [template.format(text) for text in examples["text"]]
    targets = [verbalizers[label] for label in examples["label"]]
    
    # tokenize inputs and targets
    inputs = tokenizer(texts)
    labels = tokenizer(targets)

    # concatenate inputs and labels
    for i in range(batch_size):
        sample_input_ids = inputs["input_ids"][i]
        label_input_ids = labels["input_ids"][i] + [tokenizer.pad_token_id]
        # print(i, sample_input_ids, label_input_ids)
        # be careful that the correct whitespace is between the two parts
        inputs["input_ids"][i] = sample_input_ids + label_input_ids
        # when a label is -100, the corresponding loss is ignored
        labels["input_ids"][i] = [-100] * len(sample_input_ids) + label_input_ids
        # 1 means attend to the token
        inputs["attention_mask"][i] = [1] * len(inputs["input_ids"][i])
    print(max([len(input_ids) for input_ids in inputs["input_ids"]]))

    # pad everything to max_length and convert to tensors
    for i in range(batch_size):
        sample_input_ids = inputs["input_ids"][i]
        label_input_ids = labels["input_ids"][i]
        inputs["input_ids"][i] = [tokenizer.pad_token_id] * (
            max_length - len(sample_input_ids)
        ) + sample_input_ids
        inputs["attention_mask"][i] = [0] * (max_length - len(sample_input_ids)) + inputs[
            "attention_mask"
        ][i]
        labels["input_ids"][i] = [-100] * (max_length - len(sample_input_ids)) + label_input_ids
        inputs["input_ids"][i] = torch.tensor(inputs["input_ids"][i][:max_length])
        inputs["attention_mask"][i] = torch.tensor(inputs["attention_mask"][i][:max_length])
        labels["input_ids"][i] = torch.tensor(labels["input_ids"][i][:max_length])
        
    inputs["labels"] = labels["input_ids"]
    return inputs

def tokenize_eval_examples(examples):
    # similar to tokenize_examples, but without the label

    batch_size = len(examples["text"])

    # apply template to each example
    inputs = [template.format(text) for text in examples["text"]]

    # tokenize inputs
    model_inputs = tokenizer(inputs)
    
    # pad everything to max_length and convert to tensors
    for i in range(batch_size):
        sample_input_ids = model_inputs["input_ids"][i]
        model_inputs["input_ids"][i] = [tokenizer.pad_token_id] * (
            max_length - len(sample_input_ids)
        ) + sample_input_ids
        model_inputs["attention_mask"][i] = [0] * (max_length - len(sample_input_ids)) + model_inputs[
            "attention_mask"
        ][i]
        model_inputs["input_ids"][i] = torch.tensor(model_inputs["input_ids"][i][:max_length])
        model_inputs["attention_mask"][i] = torch.tensor(model_inputs["attention_mask"][i][:max_length])
    
    out_dict = model_inputs
    out_dict["labels"] = torch.tensor(examples["label"])
    out_dict["true_labels"] = torch.tensor(examples["true_label"])
    return out_dict

Using pad_token, but it is not set yet.


In [21]:

# templateize and tokenize train
train_encodings = ds.map(
    tokenize_examples,
    batched=True,
    num_proc=1,
    remove_columns=ds["train"].column_names,
    load_from_cache_file=False,
    desc="Running tokenizer on dataset",
)

train_dataset = train_encodings["train"]

train_dataloader = DataLoader(
    train_dataset, shuffle=True, collate_fn=default_data_collator, batch_size=batch_size, pin_memory=True
)

# validation and test
eval_encodings = ds.map(
    tokenize_eval_examples,
    batched=True,
    num_proc=1,
    remove_columns=ds["train"].column_names,
    load_from_cache_file=False,
    desc="Running tokenizer on dataset",
)

eval_dataset = eval_encodings["validation"]
test_dataset = eval_encodings["test"]

eval_dataloader = DataLoader(eval_dataset, collate_fn=default_data_collator, batch_size=batch_size, pin_memory=True)
test_dataloader = DataLoader(test_dataset, collate_fn=default_data_collator, batch_size=batch_size, pin_memory=True)
print(next(iter(eval_dataloader)))
print(next(iter(test_dataloader)))

Running tokenizer on dataset:   0%|          | 0/10000 [00:00<?, ? examples/s]

Token indices sequence length is longer than the specified maximum sequence length for this model (1302 > 1024). Running this sequence through the model will result in indexing errors


1000
1378


Running tokenizer on dataset:  10%|█         | 1000/10000 [00:00<00:06, 1298.80 examples/s]

1000


...

Running tokenizer on dataset:   0%|          | 0/1000 [00:00<?, ? examples/s]               

1000
1570


Running tokenizer on dataset:   0%|          | 0/1000 [00:00<?, ? examples/s]             

1000
1372


{'input_ids': tensor([[50256, 50256, 50256,  ...,  3967,    30,   628],
        [50256, 50256, 50256,  ...,  3967,    30,   628],
        [50256, 50256, 50256,  ...,  3967,    30,   628],
        ...,
        [50256, 50256, 50256,  ...,  3967,    30,   628],
        [50256, 50256, 50256,  ...,  3967,    30,   628],
        [50256, 50256, 50256,  ...,  3967,    30,   628]]), 'attention_mask': tensor([[0, 0, 0,  ..., 1, 1, 1],
        [0, 0, 0,  ..., 1, 1, 1],
        [0, 0, 0,  ..., 1, 1, 1],
        ...,
        [0, 0, 0,  ..., 1, 1, 1],
        [0, 0, 0,  ..., 1, 1, 1],
        [0, 0, 0,  ..., 1, 1, 1]]), 'labels': tensor([1, 0, 1, 0, 0, 0, 1, 0]), 'true_labels': tensor([0, 0, 1, 0, 0, 0, 1, 0])}
{'input_ids': tensor([[    1,    39,   702,  ...,  3967,    30,   628],
        [50256, 50256, 50256,  ...,  3967,    30,   628],
        [50256, 50256, 50256,  ...,  3967,    30,   628],
        ...,
        [50256, 50256, 50256,  ...,  3967,    30,   628],
        [50256, 50256, 50256,  ...

In [33]:
from transformers import AutoModelForCausalLM
from peft import get_peft_config, get_peft_model, LoraConfig, TaskType, PeftType

peft_config = LoraConfig(
    peft_type=PeftType.LORA, task_type=TaskType.CAUSAL_LM,
    inference_mode=False, r=8, lora_alpha=32, lora_dropout=0.1
)

model = AutoModelForCausalLM.from_pretrained(model_name, torch_dtype=torch.float16, device_map={"": device})
model = get_peft_model(model, peft_config)
model = model.to(device)
model.print_trainable_parameters()

/mnt/ssd-2/spar/alexm/miniconda3/envs/dlkb/lib/python3.10/site-packages/peft/tuners/lora.py:240: UserWarning: fan_in_fan_out is set to False but the target module is `Conv1D`. Setting fan_in_fan_out to True.
  warnings.warn(


trainable params: 294912 || all params: 124734720 || trainable%: 0.23643136409814364


In [35]:
dir(model)

['T_destination',
 '__annotations__',
 '__call__',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattr__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__setstate__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_apply',
 '_backward_hooks',
 '_backward_pre_hooks',
 '_buffers',
 '_call_impl',
 '_create_repo',
 '_forward_hooks',
 '_forward_hooks_with_kwargs',
 '_forward_pre_hooks',
 '_forward_pre_hooks_with_kwargs',
 '_get_backward_hooks',
 '_get_backward_pre_hooks',
 '_get_files_timestamps',
 '_get_name',
 '_is_full_backward_hook',
 '_load_from_state_dict',
 '_load_state_dict_post_hooks',
 '_load_state_dict_pre_hooks',
 '_maybe_warn_non_full_backward_hook',
 '_modules',
 '_named_members',
 '_non_persistent_buffers_set',
 '_parameters',
 '_register

In [23]:
model.parameters

<bound method Module.parameters of PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): GPT2LMHeadModel(
      (transformer): GPT2Model(
        (wte): Embedding(50257, 768)
        (wpe): Embedding(1024, 768)
        (drop): Dropout(p=0.1, inplace=False)
        (h): ModuleList(
          (0-11): 12 x GPT2Block(
            (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
            (attn): GPT2Attention(
              (c_attn): Linear(
                in_features=768, out_features=2304, bias=True
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.1, inplace=False)
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=768, out_features=8, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=8, out_features=2304, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_emb

In [24]:
# define metrics

def logits_to_text(logits):
    ids = torch.argmax(logits[:, -1, :], dim=-1)
    return ids_to_text(ids)

def ids_to_text(ids):
    return tokenizer.batch_decode(ids, skip_special_tokens=True)
    
from sklearn.metrics import accuracy_score, f1_score

In [25]:
import numpy as np

def eval_model(use_tqdm=True):
    model.eval()
    preds = []
    labels = []
    is_erroneous = []

    iterator = tqdm(eval_dataloader) if use_tqdm else eval_dataloader
    for batch in iterator:
        with torch.no_grad():
            batch = {k: v.to(device) for k, v in batch.items()}
            outputs = model(batch["input_ids"], attention_mask=batch["attention_mask"])
            logits = outputs.logits
            text_preds = logits_to_text(logits)

            ps = [p == verbalizers[1] for p in text_preds]
            labs = batch["labels"].tolist()
            true_labs = batch["true_labels"].tolist()
            is_err = [labs[i] != true_labs[i] for i in range(len(labs))]

            preds.extend(ps)
            labels.extend(labs)
            is_erroneous.extend(is_err)
    
    preds, labels, is_erroneous = np.array(preds), np.array(labels), np.array(is_erroneous)
    acc = accuracy_score(labels, preds)
    acc_err = accuracy_score(labels[is_erroneous], preds[is_erroneous])
    acc_non_err = accuracy_score(labels[~is_erroneous], preds[~is_erroneous])
            
    return acc, acc_err, acc_non_err

acc, acc_err, acc_non_err = eval_model(use_tqdm=False)
print(f"Initial Acc: {acc}, Acc on erroneous: {acc_err}, Acc on non-erroneous: {acc_non_err}")

Initial Acc: 0.531, Acc on erroneous: 0.5714285714285714, Acc on non-erroneous: 0.5286016949152542


In [26]:
num_erroneous = 0
for row in ds["validation"]:
    if row["label"] != row["true_label"]:
        num_erroneous += 1

print(f"Number of erroneous examples in val: {num_erroneous} ({num_erroneous / len(ds['validation']) * 100:.2f}%)")

Number of erroneous examples in val: 56 (5.60%)


In [32]:
import wandb

wandb.login()

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

PermissionError: [Errno 13] Permission denied: '/tmp/.config/wandb/tmp8b5f020j.tmp'

In [28]:
from tqdm import tqdm
from torch.optim import AdamW

wandb.init(
    project="weak-deception",
        
    # track hyperparameters and run metadata
    config=config
)
# only the LORA parameters should be updated
optimizer = AdamW([p for p in model.parameters() if p.requires_grad], lr=lr, weight_decay=weight_decay)

eval_interval = 200  # steps

for epoch in range(num_epochs):
    model.train()
    total_loss = 0
    for step, batch in enumerate(tqdm(train_dataloader)):
        batch = {k: v.to(device) for k, v in batch.items()}
        outputs = model(**batch)
        loss = outputs.loss
        total_loss += loss.detach().float()
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        if (step + 1) % eval_interval == 0:
            acc, acc_err, acc_non_err = eval_model(use_tqdm=False)
            print(f"Acc: {acc}, Acc on erroneous: {acc_err}, Acc on non-erroneous: {acc_non_err}")
            wandb.log({"acc": acc, "acc_err": acc_err, "acc_non_err": acc_non_err, "loss": total_loss / step, "step": step, "epoch": epoch})
            model.train()
            
    print("Epoch {} loss: {}".format(epoch, total_loss / len(train_dataloader)))
    
wandb.finish()

  0%|          | 1/1250 [00:30<10:31:56, 30.36s/it]

Acc: 0.531, Acc on erroneous: 0.5714285714285714, Acc on non-erroneous: 0.5286016949152542


 16%|█▌        | 201/1250 [03:02<2:49:13,  9.68s/it]

Acc: 0.647, Acc on erroneous: 0.4642857142857143, Acc on non-erroneous: 0.6578389830508474


 32%|███▏      | 402/1250 [05:35<1:36:42,  6.84s/it]

...

Acc: 0.797, Acc on erroneous: 0.26785714285714285, Acc on non-erroneous: 0.8283898305084746


 96%|█████████▌| 1201/1250 [15:43<07:54,  9.67s/it]

Acc: 0.813, Acc on erroneous: 0.17857142857142858, Acc on non-erroneous: 0.850635593220339


100%|██████████| 1250/1250 [16:13<00:00,  1.28it/s]


Epoch 0 loss: 0.3303692638874054


In [31]:
# save model
# this function is overridden by the peft library
model.save_pretrained(f"custom-models/{model_name}-{ds_name}")